In [19]:
# all imports
import pandas as pd
import numpy as np
import random
from random import randint
from google.colab import drive

In [20]:
# Reaload everything from drive incase the user edited any derived KPI values.
drive.mount('/content/drive', force_remount=True)
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule", "derived_kpis"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]
derived_kpis = xls["derived_kpis"]

Mounted at /content/drive


In [21]:
# Derive the Actual count using efficiency
derived_kpis['ftt_rectified_sum'] = (derived_kpis['target_efficiency'] *
                                    derived_kpis['clock_mins'] *
                                    derived_kpis['Man power'])/ (derived_kpis['smv'] * 100)
derived_kpis['ftt_rectified_sum'] = derived_kpis['ftt_rectified_sum'].round()
derived_kpis['ftt'] = derived_kpis['ftt_rectified_sum'] * derived_kpis['ftt_ratio']
derived_kpis['ftt'] = derived_kpis['ftt'].round()
derived_kpis['rectified'] = derived_kpis['ftt_rectified_sum'] - derived_kpis['ftt']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,target_efficiency,target_defective_rate,ftt_ratio,ftt_rectified_sum,ftt,rectified
0,2024-07-26,F1M7,1,8.28,23,5,0,0,28,60,76,11,0.887614,154.0,137.0,17.0
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,89,10,0.963098,181.0,174.0,7.0
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,98,14,0.878191,199.0,175.0,24.0
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,84,4,0.863014,170.0,147.0,23.0
4,2024-07-26,F1M7,5,8.28,22,5,0,0,27,90,80,3,0.916119,235.0,215.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,76,14,0.922690,110.0,101.0,9.0
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,95,16,0.861312,138.0,119.0,19.0
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,78,8,0.846925,113.0,96.0,17.0
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,97,8,0.889287,141.0,125.0,16.0


Defective rate = (Defective  x 100 )/ (FTT + Defective + Reject)

We know that Reject pcs should be greater than or equal Defective - Rectified.

**Reject (min) =  Defective - Rectified**


We also know that the Rectified should be less than or equal to Defective.

**Defective(min) = Rectified **


Defective rate = (Defective  x 100 )/ (FTT + Defective + Defective - Rectified + Delta)

Defective goes down => numerators goes down 100 times and denominator goes down 2* => defective rate goes down. Minimum defective rate is given at minimum Defectives i.e. Defective = Rectified


Defective rate (min) = (Rectified x 100) / (FTT + 2 x Rectified - Rectified +   Delta(max))


Defective rate (min) = (Rectified x 100) / (FTT + Rectified +  Delta(max))


In [22]:
# This indicates that each line may directly reject upto 6 garments that are not
# added as defectives first
max_direct_rejects = 6
derived_kpis['direct_rejects']= np.random.randint(0, max_direct_rejects, size=len(derived_kpis))

# Implementing minimum defective rate where defectives = rectified
derived_kpis['defective_rate_min'] = np.ceil((derived_kpis['rectified']*100 )/ (derived_kpis['ftt'] + derived_kpis['rectified'] + derived_kpis['direct_rejects'])).astype(int)
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,target_efficiency,target_defective_rate,ftt_ratio,ftt_rectified_sum,ftt,rectified,direct_rejects,defective_rate_min
0,2024-07-26,F1M7,1,8.28,23,5,0,0,28,60,76,11,0.887614,154.0,137.0,17.0,4,11
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,89,10,0.963098,181.0,174.0,7.0,3,4
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,98,14,0.878191,199.0,175.0,24.0,0,13
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,84,4,0.863014,170.0,147.0,23.0,1,14
4,2024-07-26,F1M7,5,8.28,22,5,0,0,27,90,80,3,0.916119,235.0,215.0,20.0,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,76,14,0.922690,110.0,101.0,9.0,0,9
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,95,16,0.861312,138.0,119.0,19.0,0,14
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,78,8,0.846925,113.0,96.0,17.0,5,15
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,97,8,0.889287,141.0,125.0,16.0,0,12


**At this point, the programe will provide a symentically correct target_defective rate (min) value and ask the user to alter. You can go to the excel sheet and edit values. Afterwards the sheet will be reloaded and any remaining misfits will be set to the default minimum.**

In [23]:
# Find rows where targets are semantically unccceptable
for index, row in derived_kpis.iterrows():
    if row['target_defective_rate'] < row['defective_rate_min']:
        print(f"please change target_defective_rate value at row  {index} to a value larger than or equal to {row['defective_rate_min']}")

please change target_defective_rate value at row  3 to a value larger than or equal to 14
please change target_defective_rate value at row  4 to a value larger than or equal to 9
please change target_defective_rate value at row  7 to a value larger than or equal to 16
please change target_defective_rate value at row  10 to a value larger than or equal to 7
please change target_defective_rate value at row  11 to a value larger than or equal to 20
please change target_defective_rate value at row  13 to a value larger than or equal to 11
please change target_defective_rate value at row  16 to a value larger than or equal to 7
please change target_defective_rate value at row  17 to a value larger than or equal to 4
please change target_defective_rate value at row  18 to a value larger than or equal to 14
please change target_defective_rate value at row  20 to a value larger than or equal to 10
please change target_defective_rate value at row  22 to a value larger than or equal to 19
please

In [24]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='derived_kpis', index=False)



In [25]:
# Reload the excel sheet where user ay have edited target_defective-rate
drive.mount('/content/drive', force_remount=True)
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule", "derived_kpis"])

# Access individual sheets using sheet names
# plan = xls["plan"]
# schedule = xls["schedule"]
derived_kpis = xls["derived_kpis"]

Mounted at /content/drive


In [26]:
# Set any target_defective_rate values that are still incorrect to the default minimum value.
for index, row in derived_kpis.iterrows():
    if row['target_defective_rate'] < row['defective_rate_min']:
        derived_kpis.at[index, 'target_defective_rate'] = row['defective_rate_min']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,target_efficiency,target_defective_rate,ftt_ratio,ftt_rectified_sum,ftt,rectified,direct_rejects,defective_rate_min
0,2024-07-26,F1M7,1,8.28,23,5,0,0,28,60,76,11,0.887614,154,137,17,4,11
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,89,10,0.963098,181,174,7,3,4
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,98,14,0.878191,199,175,24,0,13
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,84,14,0.863014,170,147,23,1,14
4,2024-07-26,F1M7,5,8.28,22,5,0,0,27,90,80,9,0.916119,235,215,20,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,76,14,0.922690,110,101,9,0,9
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,95,16,0.861312,138,119,19,0,14
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,78,15,0.846925,113,96,17,5,15
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,97,12,0.889287,141,125,16,0,12


In [27]:
# Now derive defective, reject and defect values
derived_kpis['defective'] = (derived_kpis['target_defective_rate'] * (derived_kpis['ftt'] - derived_kpis['rectified'] + derived_kpis['direct_rejects']))/(100 - 2*derived_kpis['target_defective_rate'])
derived_kpis['defective'] = derived_kpis['defective'].round()
derived_kpis['reject'] = derived_kpis['defective'] - derived_kpis['rectified'] + derived_kpis['direct_rejects']
# derived_kpis['test'] = derived_kpis['defective'] * 100 / (derived_kpis['ftt'] + derived_kpis['defective'] + derived_kpis['reject'])
# derived_kpis['test'] = derived_kpis['test'].round()
derived_kpis['defect_increment'] = np.random.randint(0, 4, size=len(derived_kpis))
derived_kpis['defect'] = derived_kpis['defective'] + derived_kpis['defect_increment']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,ftt_ratio,ftt_rectified_sum,ftt,rectified,direct_rejects,defective_rate_min,defective,reject,defect_increment,defect
0,2024-07-26,F1M7,1,8.28,23,5,0,0,28,60,...,0.887614,154,137,17,4,11,17.0,4.0,2,19.0
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,...,0.963098,181,174,7,3,4,21.0,17.0,0,21.0
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,...,0.878191,199,175,24,0,13,29.0,5.0,3,32.0
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,...,0.863014,170,147,23,1,14,24.0,2.0,0,24.0
4,2024-07-26,F1M7,5,8.28,22,5,0,0,27,90,...,0.916119,235,215,20,1,9,22.0,3.0,3,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,...,0.922690,110,101,9,0,9,18.0,9.0,3,21.0
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,...,0.861312,138,119,19,0,14,24.0,5.0,2,26.0
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,...,0.846925,113,96,17,5,15,18.0,6.0,0,18.0
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,...,0.889287,141,125,16,0,12,17.0,1.0,3,20.0


In [28]:
# Edit plan qty and produced_mins
summed_kpis = derived_kpis.groupby(['Date', 'Line']).agg({'ftt': 'sum','rectified': 'sum','reject': 'sum'
}).reset_index()

# Merge the summed data with the plan DataFrame on 'date' and 'module_id'
plan = pd.merge(plan, summed_kpis, left_on=['date', 'module_id'], right_on=['Date', 'Line'] ,how='left')

target_increment = 20
plan['qty'] = plan['ftt'] + plan['rectified'] + plan['reject'] + np.random.randint(0, 20, size=len(plan))
plan['qty'] = plan['qty'].astype(int)
plan['produced_mins'] = plan['qty'] * plan['smv']

plan.drop(columns=['Date', 'Line', 'ftt', 'rectified', 'reject'], inplace=True)
plan['manpower'] = plan['operator'] + plan['helper'] + plan['ironman'] + plan['qc']


In [29]:
# Add TPPM
derived_kpis = derived_kpis.merge(plan[['date', 'module_id', 'qty', 'mins']],
                              left_on=['Date', 'Line'],
                              right_on= ['date','module_id'], how='left')
derived_kpis['TPPM'] = derived_kpis['qty'] / derived_kpis['mins']
derived_kpis.drop(columns=['qty', 'mins'], inplace=True)
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,rectified,direct_rejects,defective_rate_min,defective,reject,defect_increment,defect,date,module_id,TPPM
0,2024-07-26,F1M7,1,8.28,23,5,0,0,28,60,...,17,4,11,17.0,4.0,2,19.0,2024-07-26,F1M7,3.175758
1,2024-07-26,F1M7,2,8.28,23,5,0,0,28,60,...,7,3,4,21.0,17.0,0,21.0,2024-07-26,F1M7,3.175758
2,2024-07-26,F1M7,3,8.28,23,5,0,0,28,60,...,24,0,13,29.0,5.0,3,32.0,2024-07-26,F1M7,3.175758
3,2024-07-26,F1M7,4,8.28,23,5,0,0,28,60,...,23,1,14,24.0,2.0,0,24.0,2024-07-26,F1M7,3.175758
4,2024-07-26,F1M7,5,8.28,22,5,0,0,27,90,...,20,1,9,22.0,3.0,3,25.0,2024-07-26,F1M7,3.175758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,0,0,20,60,...,9,0,9,18.0,9.0,3,21.0,2024-08-01,F1M8,2.131818
150,2024-08-01,F1M8,8,8.28,14,6,0,0,20,60,...,19,0,14,24.0,5.0,2,26.0,2024-08-01,F1M8,2.131818
151,2024-08-01,F1M8,9,8.28,14,6,0,0,20,60,...,17,5,15,18.0,6.0,0,18.0,2024-08-01,F1M8,2.131818
152,2024-08-01,F1M8,10,8.28,14,6,0,0,20,60,...,16,0,12,17.0,1.0,3,20.0,2024-08-01,F1M8,2.131818


In [30]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='derived_kpis', index=False)

In [31]:
# Rename to suit the Traget KPI structure

derived_kpis.rename(columns={'smv': 'SMV', 'ftt': 'FTT', 'defective' : 'Defective', 'defect': 'Defect', 'rectified': 'Rectified', 'reject': 'Reject'}, inplace=True)

derived_kpis = derived_kpis[['Date','Line','Hour','FTT','SMV','TPPM','Defective','Defect','Rectified','Reject','Operator','Helper','Man power']]


derived_kpis['Defect'] = derived_kpis['Defect'].astype(int)
derived_kpis['Defective'] = derived_kpis['Defective'].astype(int)
derived_kpis['Reject'] = derived_kpis['Reject'].astype(int)

<ipython-input-31-8eb1392a8333>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derived_kpis['Defect'] = derived_kpis['Defect'].astype(int)
<ipython-input-31-8eb1392a8333>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derived_kpis['Defective'] = derived_kpis['Defective'].astype(int)
<ipython-input-31-8eb1392a8333>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [32]:
derived_kpis.head(50)

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Man power
0,2024-07-26,F1M7,1,137,8.28,3.175758,17,19,17,4,23,5,28
1,2024-07-26,F1M7,2,174,8.28,3.175758,21,21,7,17,23,5,28
2,2024-07-26,F1M7,3,175,8.28,3.175758,29,32,24,5,23,5,28
3,2024-07-26,F1M7,4,147,8.28,3.175758,24,24,23,2,23,5,28
4,2024-07-26,F1M7,5,215,8.28,3.175758,22,25,20,3,22,5,27
5,2024-07-26,F1M7,6,144,8.28,3.175758,39,39,12,32,23,5,28
6,2024-07-26,F1M7,7,190,8.28,3.175758,15,17,9,7,23,5,28
7,2024-07-26,F1M7,8,154,8.28,3.175758,30,30,29,3,23,5,28
8,2024-07-26,F1M7,9,137,8.28,3.175758,24,26,17,12,23,5,28
9,2024-07-26,F1M7,10,174,8.28,3.175758,19,20,19,0,23,5,28


In [33]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='target_kpis', index=False)

In [34]:
# Update the plan sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    plan.to_excel(writer, sheet_name='plan', index=False)